### Plugin setup for ImJoy

In [101]:
from imjoy import api
import zarr

def encode_zarr_store(zobj):
    def getItem(key):
        return zobj.store[key]

    def setItem(key, value):
        zobj.store[key] = value

    def containsItem(key):
        if key in zobj.store:
            return True

    return {
        "_rintf": True,
        "_rtype": 'zarr-array' if isinstance(zobj, zarr.Array) else 'zarr-group',
        "getItem": getItem,
        "setItem": setItem,
        "containsItem": containsItem,
    }

api.registerCodec({'name': 'zarr-array', 'type': zarr.Array, "encoder": encode_zarr_store})
api.registerCodec({'name': 'zarr-group', 'type': zarr.Group, "encoder": encode_zarr_store})
             
class VivPlugin:
    def __init__(self, images):
        if not isinstance(images, list):
            images = [images]
        self.images = images
    
    async def setup(self):
        pass
        
    async def run(self, ctx):
        viewer = await api.createWindow(
            type="vitessce-image-viewer-plugin", 
            src="https://epic-poitras-2b80af.netlify.app/"
        )
        for img in self.images:
            await viewer.add_image(img)
        
def run_viv(image):
    api.export(VivPlugin(image))

### Create a multiscale OME-Zarr layer

In [102]:
z_grp = zarr.open('astronaut.zarr')

image = {
    "source": z_grp, # z_group is for an ome-zarr that contains the multiscale metadata
    "channels": [
        { 
            "slider": [0, 1], 
            "color": [255, 0, 0], 
            "selection": { "c": 0 } # channel 0 is red
        },
        { 
            "slider": [0, 1], 
            "color": [0, 255, 0], 
            "selection": { "c": 1 } # channel 1 is green
        },
        { 
            "slider": [0, 1], 
            "color": [0, 0, 255], 
            "selection": { "c": 2 } # channel 2 is blue
        }
    ],
    "opacity": 1, # default is 1
    "colormap": None, # default, can be "viridis", "turbo", see viv.vitessce.io for all options
}

### Create a generic array layer

In [103]:
import numpy as np

# Need to add extra dim and cast to f32
arr = np.random.rand(1024, 1024)[None,:,:].astype('f4')
                             
noise = {
    "source": z_arr, 
    "dimensions": "cyx", # if not OME-Zarr, need to specify dimensions for array
    "channels": [
        { 
            "slider": [0, 1], 
            "color": [255, 255, 255], 
            "selection": { "c": 0 } # channel 0 is red
        },
    ],
    "opacity": 0.5,
}

### Run plugin and view layers

In [104]:
run_viv([image, noise])

<IPython.core.display.Javascript object>